In [ ]:
try:
    # settings colab:
    import google.colab
    
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/MetObjects_sample.csv "https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv?token=AA4GFHMOLN4JKDHXLJOHNKK6UGTK4"
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"


Los temas que vimos en esta parte son limpieza de datos, expresiones regulares, funciones lambda, apply .

La idea de esta práctica es ejercitar los puntos que vamos a necesitar en la clase presencial.

Usaremos el dataset de las obras del Met (The Metropolitan Museum of Art)

https://github.com/metmuseum/openaccess/



## Ejercicio 1
Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos de qué tipos de datos son las columnas.


In [ ]:
import pandas as pd

# local
data_location = "../Data/MetObjects_sample.csv"

data = pd.read_csv(data_location)

data.dtypes

## Ejercicio 2
¿Qué campos tienen valores nulos? ¿Qué porcentaje de nulos tienen cada uno de ellos?

In [ ]:
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
print(cant_nulos_por_campo)

In [ ]:
cant_registros = data.shape[0]
print(cant_registros)

porc_nulos_por_campo = 100 * cant_nulos_por_campo / cant_registros
print(porc_nulos_por_campo)

## Ejercicio 3

Analicemos la columna Object Date

¿Qué formato tienen los valores de este campo? ¿Qué patrones pueden identificar?

Sugerencia: recuerden el método value_counts
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html

In [ ]:
object_date_series = data["Object Date"]
object_date_series

Vemos que hay campos con el sufijo "th century" cuyo valor entero es un siglo, periodos de tiempo identidficados por la presencia de -, un prefijo "last quarter".

Entonces, en una misma columna tenemos años, períodos, siglos y cuartos de siglo.

## Ejercicio 4

Usando expresiones regulares, apply y funciones lambda, creemos un nuevo campo "ObjectDateClean" que sea de tipo numérico y tenga el valor del año corresponiente a cada registro extrayéndolo del campo ObjectDate.

¿Qué características tienen los valores de los registros que no verificaron ningún patrón de los definidos?

Ayuda: Identifiquemos los patrones y decidamos cómo extraer el año de los valores de los registros que verifican cada uno de esos patrones. Después, de a un patrón, usemos la estrategia que definimos antes para extraer el valor del año de los registros que lo verifican.

Vamos a empezar identificando los nulos en ese campo, que sabemos (por el resultado del ejercicio 1) son 160

In [ ]:
null_mask = object_date_series.isnull()
null_mask.sum()

Vamos a identificar ahora los enteros, que representan año. Para eso vamos a definir una expresión regular.

https://docs.python.org/3/library/re.html#match-objects

In [ ]:
import re 
import numpy as np 

year_pattern = "(\d)?\d\d\d$"
year_pattern_regex = re.compile(year_pattern)

data_year_match = object_date_series.apply(lambda x:  x if x is np.NaN else year_pattern_regex.search(x))

mask_data_year_match_notnull = data_year_match.notnull()

data.loc[mask_data_year_match_notnull, "ObjectDateClean"] = data_year_match[mask_data_year_match_notnull].apply(lambda x: x.group(0))

#len(data_year)

Veamos cómo quedaron en los valores que estaban expresados en años (sin cambios):

In [ ]:
data.loc[mask_data_year_match_notnull, ["Object Date", "ObjectDateClean"]]

Vamos a identificar ahora los períodos, quedándonos con el año desde. 

In [ ]:
period_pattern = "(?P<year_from>\d\d\d\d)\-(\d)+"
period_pattern_regex = re.compile(period_pattern)

data_period_match = object_date_series.apply(lambda x: x if x is np.NaN else period_pattern_regex.search(x))

mask_data_period_match_notnull = data_period_match.notnull()

data.loc[mask_data_period_match_notnull, "ObjectDateClean"] = data_period_match[mask_data_period_match_notnull].apply(lambda x: x.group("year_from"))


Veamos cómo quedaron en años los valores que estaban expresados en períodos:

In [ ]:
data.loc[mask_data_period_match_notnull, ["Object Date", "ObjectDateClean"]]

Vamos a identificar ahora a los siglos:

In [ ]:
century_pattern = "(?P<century>\d\d)(th (C|c)entury)"
century_pattern_regex = re.compile(century_pattern)

data_century_match = object_date_series.apply(lambda x: x if x is np.NaN else century_pattern_regex.search(x))

mask_century_period_match_notnull = data_century_match.notnull()

data_century = data_century_match[mask_century_period_match_notnull].apply(lambda x: x.group("century"))

#print(data_century)
#len(data_century)

#year_from_century = (data_century.astype(int) - 1) * 100
#year_from_century

#year_to_century = (data_century.astype(int) * 100) - 1
#year_to_century

data.loc[mask_century_period_match_notnull, "ObjectDateClean"] = \
    data_century_match[mask_century_period_match_notnull].apply(lambda x: x.group("century"))

data.loc[mask_century_period_match_notnull, "ObjectDateClean"] = \
    (data.loc[mask_century_period_match_notnull, "ObjectDateClean"].astype(int) - 1) * 100


Veamos cómo quedaron en años los valores que estaban expresados en siglos:

In [ ]:
data.loc[mask_century_period_match_notnull, ["Object Date", "ObjectDateClean"]]

Reemplazemos todos los valores nulos por 0, y verifiquemos cómo queda

In [ ]:
mask_null = object_date_series.isnull()

data.loc[mask_null, "ObjectDateClean"] = 0

data.loc[mask_null, ["Object Date", "ObjectDateClean"]]


Veamos qué características tienen los valores de los registros que nos quedaron en null en el campo ObjectDateClean

In [ ]:
mask_null_ObjectDateClean = data["ObjectDateClean"].isnull()

print(mask_null_ObjectDateClean.sum())

data.loc[mask_null_ObjectDateClean, ["Object Date", "ObjectDateClean"]]


In [ ]:
no_match_data = data["Object Date"].iloc[0]
print(ascii(no_match_data[8]))
print(ascii("-"))

Vemos que nos faltó considerar:
* los valores terminados en BC o B.C., y marcar esos años como negativos 
* considerar los períodos que tienen siglos, en lugar de años
* borrar las C. que aparecen de sufijo
* Considerar pallabras mals escritas como "cenutry"

Si tienen ganas de seguir practicando pueden resolver algunos de éstos.
